# Zhang's Algorithm For Camera Calibration

### Import Statements

In [3]:
import os
from camera_callibration_helper import *
import cv2
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


### Load the Images
* raw_img_list (list): list of 40 BGR input images
* grey_img_list (list): list of 40 grey scale input images
* img_labels (list): list of 40 image filenames (mainly for debugging)

In [4]:
# given_data_path = 'C:\\Users\jo_wang\Desktop\ECE661\HW08\Dataset1'
given_data_path = "/Users/wang3450/Desktop/ECE661/HW08/Dataset1"
raw_img_list, grey_img_list, img_labels = loadImages(given_data_path)
assert(len(grey_img_list) == 40)
assert(len(raw_img_list) == 40)
assert(len(img_labels) == 40)

x = img_labels.index('Pic_1.jpg')
y = img_labels.index('Pic_5.jpg')
z = img_labels.index('Pic_10.jpg')
w = img_labels.index('Pic_34.jpg')

print(x,y,z,w)

0 35 1 27


### Apply Canny Edge Detector On Grey Scale Images
* edge_img_list (list): list of edge maps from Canny

In [5]:
edge_img_list = performCanny(grey_img_list)
assert(len(edge_img_list) == 40)

### Apply Hough Transform To all the Images
* hough_lines_list (list): list of 40 images after applying hough transform

In [6]:
hough_lines_list = performHoughTransform(edge_img_list)
assert(len(hough_lines_list) == len(edge_img_list))

### Get the corner points from selected images
* all_corners (list): at each index, list of 80 corner points
* the_chosen_one (list): index of images to use

In [7]:
the_chosen_one = [26, 33, 29, 34]


all_corners = list()
for i in the_chosen_one:
    h_lines, v_lines = get_Horizontal_Vert_Lines(hough_lines_list[i])

    v_lines = np.array(v_lines).reshape(-1,2)
    h_lines = np.array(h_lines).reshape(-1,2)

    img = deepcopy(raw_img_list[i])
    corner_points = getCorners(v_lines, h_lines)
    if len(corner_points) == 80:
        all_corners.append(corner_points)

    for j, point in enumerate(corner_points):
        try:
            img = cv2.circle(img, point, 3, (0, 0, 255), -1)
            cv2.putText(img, str(j), (point[0]+5, point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        except OverflowError:
            pass

    cv2.imwrite(f'points_{i}.jpg', img)

### Get world point coordinates
* world_points (list): list of 80 world point coordinates in sorted order

In [8]:
world_points = list()
for i in range(0, 200, 20):
    for j in range(0, 160, 20):
        world_points.append([i,j])


### Estimate Homographies between world points and all corners
* all_homographies (list): list of 3x3 homographies relating world points to each image
* DON'T DELETE THIS ONE CUZ THIS ONE IS URS

In [9]:
all_homographies = list()
for corners in all_corners:
    h = get_H(world_points, corners)
    all_homographies.append(h)

print(all_homographies[0])

[[-1.18451284e+00 -1.51213069e+00  2.84726154e+02]
 [-1.07731089e+00 -1.18668951e+00  2.44865333e+02]
 [-4.50345703e-03 -4.61161296e-03  1.00000000e+00]]


### Compute W

In [10]:
Big_V = np.zeros((1,6))
for h in all_homographies:
    r1 = get_V(i=1, j=2, h=h).T
    r2 = get_V(i=1,j=1,h=h).T - get_V(i=2,j=2,h=h).T
    Big_V = np.vstack((Big_V, r1))
    Big_V = np.vstack((Big_V, r2))

Big_V = Big_V[1:, :]

u, s, vh = np.linalg.svd(Big_V)
b = vh[-1]

w = np.zeros((3,3))
w[0][0] = b[0]
w[0][1] = b[1]
w[0][2] = b[3]
w[1][0] = b[1]
w[1][1] = b[2]
w[1][2] = b[4]
w[2][0] = b[3]
w[2][1] = b[4]
w[2][2] = b[5]

print(w)


[[ 2.10919769e-06 -7.56829608e-07 -4.47800064e-04]
 [-7.56829608e-07  1.49040061e-05 -3.47622150e-03]
 [-4.47800064e-04 -3.47622150e-03  9.99993858e-01]]


### Compute Intrinsic Camera Parameters Matrix k

In [11]:
y0 = ((w[0][1] * w[0][2]) - (w[0][0] * w[1][2])) / (w[0][0] * w[1][1] - w[0][1] ** 2)
scale_lambda = w[2][2] - (w[0][2] ** 2 + y0 * (w[0][1] * w[0][2] - w[0][0] * w[1][2])) / w[0][0]
a_x = np.sqrt((scale_lambda / w[0][0]))
a_y = np.sqrt(((scale_lambda * w[0][0]) / (w[0][0] * w[1][1] - w[0][1] **2)))
skew = (-1 * w[0][1] * (a_x ** 2) * a_y) / scale_lambda
x0 = (skew * y0) / a_y - (w[0][2] * (a_x ** 2)) / scale_lambda

k = np.zeros((3,3))
k[0][0] = a_x
k[0][1] = skew
k[0][2] = x0
k[1][1] = a_y
k[1][2] = y0
k[2][2] = 1

print(k)

[[ 21.41751991   2.91776497 301.49406846]
 [  0.           8.13147776 248.55069932]
 [  0.           0.           1.        ]]


### Compute Extrinsic Parameters

In [12]:
#the_chosen_one = [26, 33, 29, 34]

def get_extrinsic(k, h):
    zeta = 1 / np.linalg.norm(np.linalg.inv(k) @ h[:,0])

    r1 = zeta * np.linalg.inv(k) @ h[:,0]
    r2 = zeta * np.linalg.inv(k) @ h[:,1]
    r3 = zeta * np.cross(r1,r2)
    t = zeta * np.linalg.inv(k) @ h[:,2]

    r1 = np.reshape(r1, (3,1))
    r2 = np.reshape(r2, (3,1))
    r3 = np.reshape(r3, (3,1))
    t = np.reshape(t, (3,1))

    R = np.hstack((r1,r2))
    R = np.hstack((R, r3))
    R = np.reshape(R, (3,3))

    u, _, vh = np.linalg.svd(R)

    R = u @ vh

    return R, t

all_rotations = list()
all_translations = list()

for homographies in all_homographies:
    R, t = get_extrinsic(k, homographies)
    all_rotations.append(R)
    all_translations.append(t)

assert(len(all_rotations) == len(all_translations))
assert(len(all_rotations) == len(the_chosen_one))

### Reproject the World Coordinates

In [13]:
#the_chosen_one = [26, 33, 29, 34]
def reproject(img, world_points, corners, k, r, t):
    #1). build p
    #2). apply p to world_points
    #3). plot p@world_points

    P =np.array([R[:,0].T,R[:,1].T,t.T])
    print(P.shape)

reproject(raw_img_list[26], world_points, all_corners[26], k, )

IndexError: list index out of range

In [16]:
def ReprojectPoints(img,world_coord,Corners,K,R,t):
    """
    Input: img: colored image
           world_coord: list of list of coordinates [[x1,y1],[x2,y2],...]
           corners: list of list of original coordinates of corners [[x1,y1],[x2,y2],...]
           K: Intrinsic parameter matrix 3x3
           R: Rotation matrix for this image 3x3
           t: translation vector for this image 3x1
    Output: rep_img: img with reprojected points color image
            mean_e mean of error using Euclidean distance
            var_e: variance of error using Euclidena distance
    """
    # convert world_coord to HC
    X_hc= np.ones((len(world_coord),3))
    X_hc[:,:-1]=np.array(world_coord)
    X_hc=X_hc.T # hc coordinates as col vectors
    # make camera projection matrix P
    P= np.concatenate((R[:,:2],t), axis=1)

    P=K@P
    #find reprojected points
    rep_pt_hc= P@X_hc
    # convert to physical coordinates for plotting

    rep_pt_hc = rep_pt_hc / rep_pt_hc[-1]
    rep_pt= rep_pt_hc[0:2]# physical coordinates as col vectors
    print(rep_pt.shape)
    # find Euclidean distance error, mean and var

    e=np.array(Corners).T-rep_pt
    e=np.linalg.norm(e,axis=0)
    mean_e=np.mean(e)
    var_e=np.var(e)
    # plot corners on image
    rep_img=np.copy(img)
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(world_coord)):
        rep_img=cv2.circle(img,(int(rep_pt[0,i]),int(rep_pt[1,i])),2,(0,0,255),-1)
        # rep_img=cv2.circle(img,(int(Corners[i][0]),int(Corners[i][1])),2,(0,255,0),-1)
        # rep_img=cv2.putText(img,str(i+1),(int(rep_pt[0,i]),int(rep_pt[1,i])), font,0.5,(255,0,0),1,cv2.LINE_AA)
    return(rep_img,mean_e,var_e)

In [17]:

print(len(world_points))

80


In [18]:
corner = [list(i) for i in all_corners[0]]

rep_img, _, _ = ReprojectPoints(raw_img_list[26],world_points,corner,k,all_rotations[0],all_translations[0])

cv2.imshow('test', rep_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(2, 80)
